In [ ]:
import requests
from pyquery import PyQuery
import pandas as pd
import time

def get_all_category_urls():
    url = "https://shop.vanger.com.tw/"
    res = requests.get(url)
    
    S = PyQuery(res.text)

    category_urls = S('li[href="https://shop.vanger.com.tw/men"] a').map(lambda i,e:{
        "url": PyQuery(e).attr("href"),
        "title": PyQuery(e).text(),
    }) + S('li[href="https://shop.vanger.com.tw/woman"] a').map(lambda i,e:{
        "url": PyQuery(e).attr("href"),
        "title": PyQuery(e).text(),
    }) + S('li[href="https://shop.vanger.com.tw/recommend"] a').map(lambda i,e:{
        "url": PyQuery(e).attr("href"),
        "title": PyQuery(e).text(),
    })

    category_urls_df = pd.DataFrame(category_urls)

    urls = category_urls_df.url.tolist()
    return urls 

def get_all_product_urls_in_category(category_page_url, sleep_time=10.):
    category_page_res = requests.get(category_page_url)
    S = PyQuery(category_page_res.text)

    max_page = max(S(".pagination a").map(lambda i,e: int(PyQuery(e).attr("href").replace(category_page_url+"?page=",""))))

    category_page_urls = [
        category_page_url + "?page={n}" .format(n=m) for m in range(1,max_page+1)
    ]
    
    category_page_reses = []
    
    def get_product_page_urls(category_page_res):
        S = PyQuery(category_page_res.text)
        proudct_urls = S(".product-listing .image > a").map(lambda i,e: PyQuery(e).attr("href"))
        return proudct_urls
    
    
    for category_page_url in category_page_urls:
        category_page_reses.append(get_product_page_urls(requests.get(category_page_url)))
        
        time.sleep(sleep_time)
        print("category_page_url = ",category_page_url)
        

    
    all_product_page_urls = []
    
    for urls in category_page_reses:
        all_product_page_urls.extend(urls)
        
    return all_product_page_urls
    

In [ ]:
all_category_urls = get_all_category_urls()

In [ ]:
get_all_product_urls_in_category(all_category_urls[0],2)